<a href="https://colab.research.google.com/github/FIJY/hh/blob/master/%D0%BA%D1%83%D1%81%D0%BE%D0%BA%20%D1%81%20%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3%D0%BE%D0%BC%20%D0%BE%D1%82%D0%B4%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D1%85%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Парсинг с сайта hh <br>
Ссылка на видео парсинга hh https://yandex.ru/video/preview/11468178118806870731

In [1]:
# Устанавливаем библиотеки
# requests для отправки запросов
# BeautifulSoup для разбора полученных данных
# lxml для парсинга
# fake_useragent для создания заголовков
!pip install requests
!pip install bs4
!pip install lxml
!pip install fake_useragent

In [2]:
# Импортируем библиотеки
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import json
import pandas as pd
import re

In [ ]:
# Заходим на hh, выбираем регион (Россия)
# Вводим запрос в поле поиска (например python) в разделе Соискателям
# Переходим на вкладку резюме
# Переходим на вторую страницу, чтобы появился параметр page
# Копируем адрес страницы

In [3]:
# Общий User-Agent для всех запросов
ua = fake_useragent.UserAgent()

def get_links(text, experience=None, age_from=None, age_to=None, salary=None, education=None):
    # Строим URL
    url = f"https://hh.ru/search/resume?text={text}&area=1&isDefaultArea=true&exp_period=all_time&logic=normal&pos=full_text"
    # Добавляем параметры
    if experience:
        url += f"&experience={experience}"
    if age_from:
        url += f"&age_from={age_from}"
    if age_to:
        url += f"&age_to={age_to}"
    if salary:
        url += f"&salary={salary}"
    if education:
        url += f"&education={education}"

    data = requests.get(url, headers={"user-agent": ua.random})
    if data.status_code != 200:
        print(f"Ошибка запроса: HTTP {data.status_code}")
        return []

    soup = BeautifulSoup(data.content, "lxml")

    # Определение количества страниц
    try:
        page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span", recursive=False)[-1].find("a").find("span").text)
    except Exception as e:
        print("Ошибка при получении количества страниц: ", e)
        return []

    links = []
    for page in range(1, page_count + 1):
        try:
            data = requests.get(
                url=f"{url}&page={page}",
                headers={"user-agent": ua.random}
            )
            if data.status_code != 200:
                print(f"Пропуск страницы {page} из-за ошибки HTTP {data.status_code}")
                continue
            soup = BeautifulSoup(data.content, "lxml")
            for a in soup.find_all("a", attrs={"data-qa":"serp-item__title"}):
                links.append(f"https://hh.ru{a.attrs['href'].split('?')[0]}")
        except Exception as e:
            print(f"Ошибка при обработке страницы {page}: {e}")
        time.sleep(1)
    return links


In [4]:
import csv

# Получаем ссылки
link_resume = [a for a in get_links("Python разработчик", salary="from50000")]

# Записываем полученные ссылки в CSV файл
with open('link_resume.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for resume in link_resume:
        writer.writerow([resume])  # Каждая ссылка записывается в новую строку


Пропуск страницы 250 из-за ошибки HTTP 404


In [5]:
len (link_resume)

4979

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time

# Функция для создания сессии и авторизации
def create_session(login, password):
    session = requests.Session()
    ua = fake_useragent.UserAgent()
    session.headers.update({"User-Agent": ua.random})

    login_url = "https://hh.ru/account/login"
    login_data = {'username': login, 'password': password}
    response = session.post(login_url, data=login_data)
    if response.ok:
        print("Авторизация прошла успешно")
    else:
        print("Ошибка авторизации:", response.status_code)
        session = None
    return session

# Функция для получения данных резюме
def get_resume(link):
    response = session.get(link)
    if response.status_code != 200:
        print("Ошибка доступа к странице:", response.status_code)
        return None

    soup = BeautifulSoup(response.content, 'lxml')
    try:
         gender = soup.find("span", attrs={"data-qa": "resume-personal-gender"}).text.replace("\u2009","").replace("\xa0", " ")
    except:
         gender = ""

    try:
         age = soup.find("span", attrs={"data-qa": "resume-personal-age"}).text.replace("\u2009","").replace("\xa0", " ")
    except:
         age = ""

    try:
        job_search_status = soup.find("span", attrs={"data-qa": "job-search-status"}).text.replace("\u2009","").replace("\xa0", " ")
    except:
        job_search_status = ""

  # Спарсим готовность к переезду, командировкам
    try:
      moving = soup.find(attrs={"class":"bloko-translate-guard"}).text.replace("\xa0", " ")
    except:
      moving = ""

  # Спарсим название позиции
    try:
      name = soup.find(attrs={"class":"resume-job-search-status"}).text
    except:
      name = ""
  # Спарсим название позиции

    try:
      name = soup.find(attrs={"class":"resume-block__title-text"}).text
    except:
      name = ""

  # Спарсим зарплату и уберем лишние символы (\u2009, \xa0)
    try:
      salary = soup.find(attrs={"class":"resume-block__salary"}).text.replace("\u2009","").replace("\xa0", " ")
    except:
      salary = ""

  # Спарсим специализации и график работы
    try:
      specialization_schedule = re.sub(r'([А-ЯA-Z])', r' \1', soup.find(attrs={"class":"resume-block-container"}).text).strip()
    except:
      specialization_schedule = ""

  # Спарсим опыт работы
    try:
        experience = soup.find(attrs={"data-qa":"resume-block-experience"}).text.replace("\xa0", " ")
    except:
        experience = ""

  # Находим атрибут, где находятся все ключевые навыки
    try:
      tags = [tag.text for tag in soup.find(attrs={"data-qa":"skills-table"}).find_all(attrs={"class":"bloko-tag__section_text"})]
    except:
      tags = []

  # Добавляем линк в словарь
    resume = {
      # "persons": persons,
      "gender": gender,
      "age": age,
      "job_search_status":job_search_status,
      "moving": moving,
      "name": name,
      "salary": salary,
      "specialization_schedule": specialization_schedule,
      "experience": experience,
      "tags": tags,
      "link": link
    }
    return resume



# Функция для обработки файлов и ссылок
# def process_resumes(filename, session):
#     links = []
#     # Чтение ссылок из файла CSV
#     with open(filename, newline='', encoding='utf-8') as csvfile:
#         reader = csv.reader(csvfile)
#         links = [row[0] for row in reader]  # Предполагается, что ссылки находятся в первом столбце

#     for link in links[:10]:  # ограничим количество для примера
#         resume_data = get_resume(link)
#         print(resume_data)
#         time.sleep(1)  # Искусственная задержка между запросами
def process_resumes(filename, session):
    links = []
    # Чтение ссылок из файла CSV
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        links = [row[0] for row in reader]  # Предполагается, что ссылки находятся в первом столбце

    # Создаем файл CSV для записи результатов
    with open('/content/drive/MyDrive/Practice2/resume_links/IT_age_0_19_output.csv', 'w', newline='', encoding='utf-8') as output_file:
        fieldnames = ['gender', 'age', 'job_search_status', 'moving', 'name', 'salary', 'specialization_schedule', 'experience', 'tags', 'link']
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()  # Записываем заголовки столбцов

        for link in links[:]:  # ограничим количество для примера
            resume_data = get_resume(link)
            if resume_data:
                writer.writerow(resume_data)  # Записываем данные в CSV
            print(resume_data)
            time.sleep(1)  # Искусственная задержка между запросами

# Параметры и запуск функций аналогичны предыдущему примеру
# Параметры
login = "sivilio@sivilio.by"
password = "Sivilio4UII"
base_url = "https://hh.ru/search/resume?exp_period=all_time&filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&pos=full_text&search_period=30&items_on_page=20&filter_exp_period=all_time&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=0&age_to=19"
filename = "/content/drive/MyDrive/Practice2/resume_links/IT_age_0_19.csv"

session = create_session(login, password)
if session:
    process_resumes(filename, session)


Авторизация прошла успешно
{'gender': 'Мужчина', 'age': '19 лет', 'job_search_status': 'Рассматривает предложения', 'moving': 'Москва, не готов к переезду, не готов к командировкам', 'name': 'Программист C#', 'salary': '', 'specialization_schedule': 'Специализации: Программист, разработчик Занятость: полная занятость, частичная занятость, стажировка График работы: полный день, сменный график, гибкий график, удаленная работа', 'experience': "Опыт работы 1 год 8 месяцев Октябрь 2023 — по настоящее время8 месяцевШкола программирования 21 от СБЕРМосква, applicant.21-school.ru/Информационные технологии, системная интеграция, интернет... Показать ещеПрограммистИзучение C/C++/C#/SQL, обучение навыкам devops инженера Linux server, Docker, CI/CD, NAT сеть, Сетевой мост, протоколы разных уровней.\nСделал более 12 проектов(SimpleBashUtilites, Decimal, Docker, Linux, Docker, Matrix)Октябрь 2022 — по настоящее время1 год 8 месяцевЛабораторный практикум в ВУЗеМоскваПрограммист C#Изучал web и desktop

In [ ]:
# Извлекаем данные из файла .csv с помощью pandas
data_df = pd.read_csv("data.csv")
data_df.head()

,persons,moving,name,salary,specialization_schedule,experience,tags,link
0,"['Мужчина', '24 года', '10 декабря 1999', 'Нов...","Новосибирск, м. Заельцовская, готов к переезду...",Python developer,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 4 месяца Март 2023 — по нас...,"['Git', 'Python', 'ООП', 'Django Framework', '...",https://hh.ru/resume/7b1a872100086178440039ed1...
1,"['Мужчина', '32 года', '6 января 1992', 'Москв...","Москва, м. Тульская, готов к переезду, готов к...",Программист Python,350000 ₽ на руки,350 000 ₽ на руки,Опыт работы 8 лет 9 месяцев Ноябрь 2016 — по н...,"['Python', 'Machine Learning', 'Linux', 'Compu...",https://hh.ru/resume/fb7ea8c800035e24280039ed1...
2,"['Мужчина', '23 года', '23 июня 2000', 'Москва...","Москва, м. Сходненская, не готов к переезду, г...",Python-разработчик/ Python developer,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 4 года 2 месяца Декабрь 2020 — по ...,"['Исполнительность', 'Поиск информации в интер...",https://hh.ru/resume/6a9abec70008de01cc0039ed1...
3,"['Мужчина', '24 года', '17 февраля 2000', 'Мос...","Москва, не готов к переезду, не готов к команд...",Программист,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 3 года 4 месяца Июль 2023 — по нас...,"[' Python', 'JavaScript', 'Git', 'Bash', 'Linu...",https://hh.ru/resume/bee0660c00080a04f10039ed1...
4,"['Мужчина', '31 год', '28 февраля 1993', 'Влад...","Владивосток, готов к переезду (Беларусь, Москв...",Программист Python,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 7 лет 8 месяцев Сентябрь 2016 — по...,[],https://hh.ru/resume/b20787e00003b6474f0039ed1...
